In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.tools import tool
import requests
import os
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate great-circle distance between two points on Earth."""
    R = 6371  # Earth radius in kilometers
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)
    a = math.sin(d_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

@tool
def get_hotels_by_area_and_radius(
    bbox: str,
    arrival_date: str,
    departure_date: str,
    star_rating: str = "3,4,5",
    room_qty: int = 1,
    guest_qty: int = 1,
    children_qty: int = 0,
    children_age: str = "",
    currency: str = "USD",
    order_by: str = "popularity",
    categories_filter: str = "class::1,class::2,class::3",
    language: str = "en-us",
    travel_purpose: str = "leisure",
    offset: int = 0
) -> list:
    """
    Fetch hotel listings within a bounding box and return only key fields, sorted by distance to bbox center.

    Parameters:
        - bbox (str): Bounding box in format "min_lat,max_lat,min_lng,max_lng"
        - star_rating (str): Comma-separated star classes to filter, e.g., "3,4,5"
        - arrival_date (str): Check-in date (YYYY-MM-DD)
        - departure_date (str): Check-out date (YYYY-MM-DD)
        - room_qty (int): Number of rooms
        - guest_qty (int): Number of adults
        - children_qty (int): Number of children
        - children_age (str): Comma-separated list of children ages
        - currency (str): Price currency (e.g., USD, INR)
        - order_by (str): API sort preference (not used post-filter)
        - categories_filter (str): Used internally, overridden by star_rating
        - language (str): Response language
        - travel_purpose (str): "leisure" or "business"
        - offset (int): Pagination offset

    Returns:
        - list of hotel dicts sorted by ascending distance from bbox center.
    """

    # Convert star_rating to API-compatible format
    categories_filter = ",".join([f"class::{s.strip()}" for s in star_rating.split(",")])

    # Compute bbox center
    try:
        min_lat, max_lat, min_lng, max_lng = map(float, bbox.split(","))
        center_lat = (min_lat + max_lat) / 2
        center_lng = (min_lng + max_lng) / 2
    except Exception as e:
        return [{"error": f"Invalid bbox format: {e}"}]

    url = "https://apidojo-booking-v1.p.rapidapi.com/properties/list-by-map"
    querystring = {
        "room_qty": str(room_qty),
        "guest_qty": str(guest_qty),
        "bbox": bbox,
        "search_id": "none",
        "children_age": children_age,
        "price_filter_currencycode": currency,
        "categories_filter": categories_filter,
        "languagecode": language,
        "travel_purpose": travel_purpose,
        "children_qty": str(children_qty),
        "order_by": order_by,
        "offset": str(offset),
        "arrival_date": arrival_date,
        "departure_date": departure_date
    }
    headers = {
        "x-rapidapi-key": os.getenv("RAPIDAPI_KEY_HOTELS"),
        "x-rapidapi-host": "apidojo-booking-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code != 200:
        return [{"error": response.text}]

    results = response.json().get("result", [])
    hotels = []

    for item in results:
        if not item.get("class"):
            continue

        lat = item.get("latitude")
        lng = item.get("longitude")
        if lat is None or lng is None:
            continue

        distance_km = haversine_distance(center_lat, center_lng, lat, lng)

        hotels.append({
            "name": item.get("hotel_name"),
            "star_rating": item.get("class"),
            "review_score": item.get("review_score"),
            "review_word": item.get("review_score_word"),
            "review_count": item.get("review_nr"),
            "address": item.get("address"),
            "city": item.get("city"),
            "district": item.get("district"),
            "latitude": lat,
            "longitude": lng,
            "price_per_night": item.get("min_total_price") or (
                item.get("price_breakdown", {}).get("all_inclusive_price")
            ),
            "image": item.get("main_photo_url"),
            "booking_url": item.get("url"),
            "is_free_cancellable": item.get("is_free_cancellable"),
            "is_mobile_deal": item.get("is_mobile_deal"),
            "checkin_from": item.get("checkin", {}).get("from"),
            "checkout_until": item.get("checkout", {}).get("until"),
            "distance_km": round(distance_km, 2)
        })

    # Sort by distance from bbox center
    hotels.sort(key=lambda h: h["distance_km"])
    return hotels

In [3]:
# %pip install geopy

In [4]:
from geopy.geocoders import Nominatim

In [5]:
def get_coordinates(location: str):
    geolocator = Nominatim(user_agent="bbox_converter_app")
    location_obj = geolocator.geocode(location)
    if location_obj:
        return location_obj.latitude, location_obj.longitude
    else:
        raise ValueError("Could not geocode location.")

In [6]:
get_coordinates("Park Street, Kolkata")

(22.5551591, 88.3501171)

In [7]:
get_coordinates("Koramangala, Bangalore")

(12.9385701, 77.6329963)

In [8]:
get_coordinates("South bangalore")

(12.9881567, 77.6226)

In [9]:
def get_bbox(lat, lon, radius_km=5):
    lat_delta = radius_km / 111  # 1 deg lat ~ 111 km
    lon_delta = radius_km / (111 * abs(math.cos(math.radians(lat))) or 0.0001)  # Avoid division by zero
    return f"{lat - lat_delta:.4f},{lat + lat_delta:.4f},{lon - lon_delta:.4f},{lon + lon_delta:.4f}"


In [10]:
lat, lon = get_coordinates("South bangalore")

get_bbox(lat, lon)

'12.9431,13.0332,77.5764,77.6688'

In [11]:
def location_to_bbox_string(location: str, radius_km=5):
    lat, lon = get_coordinates(location)
    return get_bbox(lat, lon, radius_km)

In [17]:
location_to_bbox_string("south goa")

'15.1746,15.2647,74.0692,74.1626'

In [18]:
bbox = location_to_bbox_string("south goa")
response = get_hotels_by_area_and_radius.invoke({
    "bbox": bbox,
    "arrival_date": "2025-07-13",
    "departure_date": "2025-07-15",
    "star_rating": "4,5"
})

response

[]

In [19]:
len(response)

0